# Data Acquisition

In [1]:
import pandas as pd
import numpy as np

import env
from acquire import get_zillow_data

Document filter specificationsfor project

Reminder: Filter lasttransactiondate for May and June ONLY

In [7]:
df = get_zillow_data()

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (36,37,62,63,66,67) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
get_zillow_data??

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77575 entries, 0 to 77574
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        222 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77575 non-null  float64
 3   heatingorsystemtypeid         49570 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      206 non-null    float64
 6   airconditioningtypeid         25006 non-null  float64
 7   parcelid                      77575 non-null  int64  
 8   id                            77575 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77575 non-null  float64
 11  bedroomcnt                    77575 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c